In [ ]:
# Import dependancies




# Import API key
from api_keys import weather_api_key

In [ ]:
#  Generate Cities List

In [ ]:
#  Perform API Calls

# Perform a weather check on each city using a series of successive API calls.
# Include a print log of each city as it'sbeing processed (with the city number and city name).

In [ ]:
#  Convert Raw Data to DataFrame
# Export the city data into a .csv.
# Display the DataFrame

In [ ]:
#  Inspect the data and remove the cities where the humidity > 100%.
# Skip this step if there are no cities that have humidity > 100%.

In [ ]:
#  Get the indices of cities that have humidity over 100%.

In [ ]:
 # Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

Plotting the data

In [ ]:
# Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.Save the plotted figures as .pngs.

In [ ]:
# Latitude vs. Temperature Plot

In [ ]:
#  Latitude vs. Humidity Plot

In [ ]:
#  Latitude vs. Cloudiness Plot

In [ ]:
# Latitude vs. Wind Speed Plot

Linear Regression

In [ ]:
#  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression